# Filtracja bilateralna

## Konwolucja obrazu z filtrem o zadanych współczynnikach

Splot (konwolucję) obrazu wejściowego $I$ z filtrem $\psi$ dla ustalonego punktu obrazu $\mathbf{x}$ można przedstawić następująco:

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||)I(\mathbf{p})
\end{equation}

gdzie:
- $\hat{I}$ jest obrazem wynikowym (przefiltrowanym),
- $W_N = \sum_y \psi(y)$ jest parametrem normalizującym współczynniki filtra $\psi$,
- $||\cdot||$ jest odległością między punktami obrazu $\mathbf{x}$ i $\mathbf{p}$ według ustalonej metryki (np. norma $L_2$). Uwaga, proszę pamiętać, że zarówno $\mathbf{x}$, jak i $\mathbf{p}$ to współrzędne przestrzenne,
- $\eta(\mathbf{x})$ jest otoczeniem punktu $\mathbf{x}$.

Funkcja $\psi$ decyduje o charakterze filtracji. Dla filtru Gaussowskiego:

\begin{equation}
\psi(y) = G_{\delta_s}(y)
\end{equation}

gdzie: $G_{\delta_s}(y)$ jest funkcją Gaussa z parametrem skali $\delta_s$.

Opisaną powyżej filtrację realizowaliśmy w ramach ćwiczenia "Przetwarzanie wstępne. Filtracja kontekstowa."

## Filtracja bilateralna

Wadą klasycznego splotu jest brak adaptacji współczynników filtra do lokalnego otoczenia $\eta(\mathbf{x})$ filtrowanego punktu $\mathbf{x}$.
Oznacza to wykorzystanie tych samych współczynników filtra $\psi$ niezależnie od tego czy otoczenie jest względnie jednorodne lub zawiera krawędzie obiektów (w tym przypadku dochodzi do "rozmywania" krawędzi).
Filtracja bilateralna uwzględnia lokalne otoczenie filtrowanego punktu, w ten sposób, że parametry filtra zmieniają się w zależności od "wyglądu" otocznia.


Współczynniki filtra obliczane są na podstawie odległości filtrowanego punktu $\mathbf{x}$ od każdego punktu otoczenia $\mathbf{p}$ w dziedzinie przestrzennej obrazu (tak jak przy typowym filtrze np. Gaussa) oraz odległości punktów w przeciwdziedzinie obrazu (np. na podstawie różnicy w jasności pikseli dla obrazu w odcieniach szarości):

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||) \gamma(|I(\mathbf{p}) - I(\mathbf{x})|) I(\mathbf{p})
\end{equation}
gdzie:
- $W_N$ jest współczynnikiem normalizującym filtr,
- $\gamma$ jest funkcją odległości w przeciwdziedzinie obrazu, np. $\gamma(y)=\exp(-\frac{y^2}{2\delta_r^2})$
- parametr $\delta_r$ jest utożsamiany z poziomem szumu w obrazie i należy go dobrać w sposób empiryczny.

Proszę chwilę zastanowić się nad powyższym równaniem, w szczególności nad funkcją $\gamma$. Proszę wyznaczyć, jaka będzie wartość funkcji dla pikseli podobnych (różnica 0, 1, 2), a skrajnie różnych (255, 200).

##  Realizacja ćwiczenia

### Wczytanie danych

1. Wczytaj dane z pliku *MR_data.mat*. W tym celu wykorzystaj funkcję `loadmat` z pakietu scipy:
        from scipy.io import loadmat
        mat = loadmat('MR_data.mat')

2. Wczytany plik zawiera 5 obrazów: *I_noisefree*, *I_noisy1*, *I_noisy2*, *I_noisy3* oraz *I_noisy4*. Odczytać je można w następujący sposób:
        Input = mat['I_noisy1']

3.Wyświetl wybrany obraz z pliku *MR_data.mat*. Zagadka - co to za obrazowanie medyczne?

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal
from scipy.io import loadmat
import math

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/'

fileNames = ["MR_data.mat"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)

#TODO Samodzielna
mat = loadmat('MR_data.mat')
Input = mat['I_noisy1']

plt.imshow(Input, 'gray')
plt.axis('off')
plt.show()

#MRI

### "Klasyczna" konwolucja

1. Zdefiniuj parametry filtra Gaussowskiego: rozmiar okna i wariancję $\delta_S$.
2. Oblicz współczynniki filtra na podstawie zdefiniowanych parametrów (najprościej w ramach podwójnej pętli for).
2. Sprawdź ich poprawność i zwizualizuj filtr (tak jak w ćwiczeniu pt. "Przetwarzanie wstępne. Filtracja kontekstowa.").
3. Wykonaj kopię obrazu wejściowego: `IConv = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentem tej funkcji są aktualnie przetwarzane okno i współczynniki filtra.
7. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
8. Wyświetl wynik filtracji.
9. Porównaj wynik z obrazem oryginalnym.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

window_size = 15
delta_s = 0.8

#wspolczynniki
def compute_gaussian_filter(size, sigma):
    kernel = np.zeros((size, size), dtype=np.float32)
    offset = size // 2

    for i in range(size):
        for j in range(size):
            x = i - offset
            y = j - offset
            kernel[i, j] = np.exp(-(x**2 + y**2) / (2 * sigma**2)) / (2 * np.pi * sigma**2)
    return kernel / np.sum(kernel)

def fgaussian(size, sigma):
    m = n = size
    h, k = m//2, n//2
    x, y = np.mgrid[-h:h+1, -k:k+1]
    g = np.exp(-(x**2 + y**2)/(2*sigma**2))
    return g /g.sum()


def mesh(fun, size):
    fig = plt.figure()
    ax = fig.add_subplot(projection = '3d')


    X = np.arange(-size//2, size//2, 1)
    Y = np.arange(-size//2, size//2, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun

    ax.plot_surface(X, Y, Z)

    plt.show()


#print(compute_gaussian_filter(window_size, delta_s))
#print(fgaussian(window_size, delta_s))


mesh(fgaussian(window_size, delta_s),window_size)
mesh(compute_gaussian_filter(window_size, delta_s),window_size)

In [ ]:
IConv = Input.copy()
X,Y = IConv.shape

contexts = []

for x in range(window_size//2, X - window_size//2):
  for y in range(window_size//2, Y - window_size//2):
    current_context = IConv[x-window_size//2:x+window_size//2, y-window_size//2: y+window_size//2]
    contexts.append(current_context)

print(contexts[0])

In [ ]:
def gaussian_convolution(context, window_size, delta_s):
    X, Y = context.shape
    gauss = compute_gaussian_filter(window_size, delta_s)
    output = 0
    for i in range(X):
        for j in range(Y):
            output += gauss[i, j] * context[i, j]
    return 1 / gauss.sum() * output

In [ ]:
def gaussian_blur(image, window_size, delta_s):
  X,Y = image.shape

  new_image = image.copy()

  for x in range(window_size//2, X - window_size//2):
    for y in range(window_size//2, Y - window_size//2):
      current_context = image[x-window_size//2:x+window_size//2, y-window_size//2: y+window_size//2]
      new_image[x,y] = gaussian_convolution(current_context, window_size, delta_s)

  return new_image

In [ ]:
window_size = 5
delta_s = 1


f,axs=plt.subplots(1,2,figsize=(20,10))
axs[0].imshow(Input,'gray')
axs[0].set_title('Obraz oryginalny')
axs[1].imshow(gaussian_blur(Input, window_size, delta_s),'gray')
axs[1].set_title('Obraz po filtracji')
for ax in axs:
  ax.axis('off')

### Filtracja bilateralna

1. Zdefiniuj dodatkowy parametr: wariancję $\delta_R$.
3. Wykonaj kopię obrazu wejściowego: `IBilateral = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentami funkcji są aktualnie przetwarzane okno, współczynniki filtra gausowskiego (takie same jak wcześniej) i wariancja $\delta_R$.
7. Oblicz odległość w przeciwdziedzinie (dla wartości pikseli).
8. Oblicz funkcję Gaussa dla obliczonych odległości z zadanym parametrem.
9. Wykonaj normalizację obliczonych współczynników.
10. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
11. Wyświetl wynik filtracji.
12. Porównaj wynik z obrazem oryginalnym.

In [ ]:
def new_pixel_value(context, kernel, central_pixel, sigmaR):
    range_kernel = np.exp(-0.5 * ((context - central_pixel)**2) / (sigmaR**2))
    combined_weights = kernel * range_kernel
    new_value = np.sum(context * combined_weights) / np.sum(combined_weights)

    return new_value

In [ ]:
sigmaR = 30
IBilateral = Input.copy()
kernel = compute_gaussian_filter(5, 1)
half_size = kernel.shape[0] // 2

for i in range(window_size//2, Input.shape[0] - window_size//2):
    for j in range(window_size//2, Input.shape[1] - window_size//2):
        context = Input[i - window_size//2:i + window_size//2 + 1, j - window_size//2:j + window_size//2 + 1]
        central_pixel = Input[i, j]
        IBilateral[i, j] = new_pixel_value(context, kernel, central_pixel, sigmaR)

plt.figure(figsize=(15, 15))
plt.subplot(1, 2, 1)
plt.title("Oryginał")
plt.imshow(Input, cmap='gray')
plt.subplot(1, 2, 2)
plt.title("Po filtracji")
plt.imshow(IBilateral, cmap='gray')
plt.show()